In [1]:
import numpy as np 
import glob 
import matplotlib.pyplot as plt 


def plot_fbank(fbank, title=None, save_path=None, **kwargs):
    fig, axs = plt.subplots(min(4, fbank.shape[0]), 1, sharex=True, sharey=True)
    vmin, vmax = kwargs.get("vmin", None), kwargs.get("vmax", None)
    cbar = None
    # max 4 channels...
    for channel in range(0, min(4, fbank.shape[0])):
        axs[channel].set_title(f"Filter bank channel {channel}, {title}")

        if fbank.shape[0] == 4 and channel >= 2:
            im = axs[channel].imshow(fbank[channel].T, aspect="auto")
        else:
            im = axs[channel].imshow(fbank[channel].T, aspect="auto", vmin=vmin, vmax=vmax)
            cbar = im 
        axs[channel].set_ylabel("mel")
        axs[channel].set_xlabel("time")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    fig.colorbar(cbar, ax=axs.ravel().tolist())
    plt.show()
    if save_path:
        fig.savefig(save_path)
    plt.close()
    return fig


from torch import nn
import torch 

norm = nn.LayerNorm(
                    [7, 6001, 128], 
                    elementwise_affine=False
                )
gram_path = "/projects/0/prjs1338/STARSS2023_gram_labels/foa_dev"
normal_path = "/projects/0/prjs1338/STARSS2023_labels/foa_dev"
files = sorted(glob.glob(f"{gram_path}/*.npy"))
for file in files:
    print(file)
    spec = np.load(file)
    from einops import rearrange 
    spec = rearrange(spec, "T (C F) -> C T F", F =128, C=7)
    spec = norm(torch.tensor(spec))
    plot_fbank(spec[:, :2000, :])
    break
    
